# 🧠 NSAI - Neurosymbolic AI Runner Selection

> Interactive Demo & Playground for the Symbolic Layer

**ADR:** AI-001 | **Epic:** #27 | **Paper:** #26 (JKU Bachelor)

---

## Architecture Overview

```
┌─────────────────────────────────────────────────────┐
│  SYMBOLIC LAYER (This Notebook)                     │
│  ├── ontology/   Runner Capability Ontology         │
│  ├── parser/     Job Requirement Parser             │
│  └── csp/        Constraint Satisfaction Solver     │
│                       ↓                             │
│              [Feasible Runner Set]                  │
├─────────────────────────────────────────────────────┤
│  SUBSYMBOLIC LAYER (runner_bandit service)          │
│  └── Multi-Armed Bandit Selection (UCB1, Thompson)  │
│                       ↓                             │
│              [Selected Runner]                      │
└─────────────────────────────────────────────────────┘
```

## 🔧 Setup

In [ ]:
# Colab Setup (uncomment if running in Colab)
# !pip install pyyaml networkx matplotlib --quiet
# !git clone https://github.com/blauweiss-llc/backoffice.git /content/backoffice
# import sys; sys.path.insert(0, '/content/backoffice/services')

# Local Setup
import sys
from pathlib import Path

# Add parent to path for imports
notebook_dir = Path('.').absolute()
if 'services/nsai' in str(notebook_dir):
    sys.path.insert(0, str(notebook_dir.parent.parent))
else:
    sys.path.insert(0, str(notebook_dir.parent))

print(f"Python: {sys.version}")
print(f"Working dir: {notebook_dir}")

In [ ]:
# Core imports
from nsai.ontology import RunnerOntology
from nsai.ontology.runner_ontology import (
    Runner, RunnerCapability, CapabilityType, create_blauweiss_ontology
)
from nsai.parser import JobRequirementParser, JobRequirements
from nsai.csp import ConstraintSolver
from nsai.csp.constraint_solver import SolverStatus

# Visualization
import json
from pprint import pprint

print("✅ NSAI modules loaded successfully!")

---

## 1️⃣ Ontology - Runner Capabilities

The ontology defines **what runners can do** using a capability taxonomy.

In [ ]:
# Create a fresh ontology
onto = RunnerOntology()

# Show built-in capability types
print("📚 Capability Types:")
for cap_type in CapabilityType:
    print(f"  • {cap_type.name}")

print(f"\n📋 Standard Capabilities: {len(onto.STANDARD_CAPABILITIES)}")
for name, ctype in list(onto.STANDARD_CAPABILITIES.items())[:10]:
    print(f"  • {name:15} → {ctype.name}")
print("  ...")

In [ ]:
# Add runners with capabilities
onto.add_runner(
    name="gitlab-runner-nordic",
    runner_id=12345,
    capabilities=["docker", "shell", "gcp", "nordic", "linux"],
    tags=["docker-any", "shell", "nordic"],
    cost_per_minute=0.01
)

onto.add_runner(
    name="gpu-runner-aws",
    capabilities=["docker", "gpu", "aws", "linux"],
    tags=["docker", "gpu"],
    cost_per_minute=0.50  # GPUs are expensive!
)

onto.add_runner(
    name="mac-local",
    capabilities=["shell", "macos"],
    tags=["shell", "macos"],
    cost_per_minute=0.0,
    online=False  # Currently offline
)

onto.add_runner(
    name="k8s-cluster",
    capabilities=["kubernetes", "docker", "linux", "azure"],
    tags=["k8s", "azure"],
    cost_per_minute=0.05
)

print(f"✅ Added {len(onto.runners)} runners")
for name, runner in onto.runners.items():
    caps = [c.name for c in runner.capabilities]
    status = "🟢" if runner.online else "🔴"
    print(f"  {status} {name}: {caps}")

In [ ]:
# Query the ontology
print("🔍 Runners with 'docker' capability:")
for r in onto.get_runners_with_capability("docker"):
    print(f"  • {r.name}")

print("\n🔍 Runners with 'docker' AND 'linux':")
for r in onto.get_runners_with_all_capabilities(["docker", "linux"]):
    print(f"  • {r.name}")

print("\n🔍 Feasible runners (require docker, exclude gpu):")
for r in onto.get_feasible_runners(required=["docker"], excluded=["gpu"]):
    print(f"  • {r.name} (cost: €{r.cost_per_minute}/min)")

### 🎨 Capability Implications

Some capabilities **imply** others (e.g., `nordic` → `gcp`, `eu-west`)

In [ ]:
print("🔗 Capability Implications:")
for cap, implied in onto.CAPABILITY_IMPLICATIONS.items():
    print(f"  {cap} → {implied}")

# Check nordic runner - should have implied capabilities
nordic = onto.runners["gitlab-runner-nordic"]
print(f"\n📋 Nordic runner capabilities (with implications):")
for cap in sorted(nordic.capabilities, key=lambda c: c.name):
    print(f"  • {cap.name}")

---

## 2️⃣ Parser - Job Requirements

The parser extracts **what jobs need** from `.gitlab-ci.yml` definitions.

In [ ]:
parser = JobRequirementParser()

# Simple job with tags
job1 = {
    "tags": ["docker-any"],
    "image": "python:3.11",
    "script": ["pytest"]
}

reqs1 = parser.parse(job1, "unit-tests")
print("📋 Job: unit-tests")
print(f"  Required:  {reqs1.required_capabilities}")
print(f"  Preferred: {reqs1.preferred_capabilities}")
print(f"  Tags:      {reqs1.tags}")

In [ ]:
# GPU job with NVIDIA image
job2 = {
    "tags": ["docker", "gpu"],
    "image": "nvidia/cuda:11.8-base",
    "timeout": "2h 30m",
    "script": ["python train.py"]
}

reqs2 = parser.parse(job2, "ml-training")
print("📋 Job: ml-training")
print(f"  Required:  {reqs2.required_capabilities}")
print(f"  Preferred: {reqs2.preferred_capabilities}")
print(f"  Timeout:   {reqs2.timeout_seconds}s ({reqs2.timeout_seconds/60:.0f} min)")

In [ ]:
# Parse complete .gitlab-ci.yml
yaml_content = """
default:
  tags:
    - docker-any

stages:
  - test
  - build
  - deploy

unit-tests:
  stage: test
  image: python:3.11
  script:
    - pytest tests/

build-image:
  stage: build
  tags:
    - docker-any
    - gcp
  services:
    - docker:dind
  script:
    - docker build -t myapp .

deploy-k8s:
  stage: deploy
  tags:
    - k8s
  script:
    - kubectl apply -f k8s/
"""

jobs = parser.parse_yaml(yaml_content)
print(f"📋 Parsed {len(jobs)} jobs from YAML:\n")
for name, reqs in jobs.items():
    print(f"  {name}:")
    print(f"    Required: {reqs.required_capabilities}")
    if reqs.preferred_capabilities:
        print(f"    Preferred: {reqs.preferred_capabilities}")

---

## 3️⃣ CSP Solver - Finding Feasible Runners

The solver combines ontology + requirements to find **which runners can execute a job**.

In [ ]:
# Create solver with our ontology
solver = ConstraintSolver(ontology=onto, parser=parser)

# Solve for a simple Docker job
result = solver.solve({"tags": ["docker-any"]}, "simple-job")

print(f"🎯 Result for 'simple-job':")
print(f"  Status: {result.status.name}")
print(f"  Feasible: {result.feasible_runners}")
print(f"  Best: {result.best_runner}")
print(f"  Solve time: {result.solve_time_ms:.2f}ms")
print(f"\n📝 Explanation:\n{result.explanation}")

In [ ]:
# Solve for GPU job - should prefer gpu-runner
result_gpu = solver.solve(
    {"tags": ["docker"], "image": "nvidia/cuda:11.8"},
    "gpu-training"
)

print(f"🎯 Result for 'gpu-training':")
print(f"  Ranked runners:")
for name, score in result_gpu.ranked_runners:
    runner = onto.runners[name]
    print(f"    {name}: score={score:.2f}, cost=€{runner.cost_per_minute}/min")

In [ ]:
# Infeasible job - no runner can do this
result_impossible = solver.solve(
    {"tags": ["quantum-computer", "docker"]},  # We don't have a quantum computer!
    "quantum-job"
)

print(f"🎯 Result for 'quantum-job':")
print(f"  Status: {result_impossible.status.name}")
print(f"  Feasible: {result_impossible.feasible_runners}")
print(f"\n📝 Explanation:\n{result_impossible.explanation}")

In [ ]:
# Batch solve for all jobs in a pipeline
pipeline_jobs = {
    "lint": {"tags": ["docker-any"], "image": "python:3.11"},
    "test": {"tags": ["docker-any"], "image": "python:3.11"},
    "build": {"tags": ["docker-any", "gcp"]},
    "deploy": {"tags": ["k8s"]},
    "train-ml": {"tags": ["docker", "gpu"], "image": "nvidia/cuda:11.8"}
}

results = solver.solve_batch(pipeline_jobs)

print("📋 Pipeline Feasibility Matrix:\n")
print(f"{'Job':<15} {'Status':<12} {'Best Runner':<25} {'Alternatives'}")
print("-" * 70)
for job_name, result in results.items():
    status = "✅" if result.is_feasible else "❌"
    best = result.best_runner or "NONE"
    alts = len(result.feasible_runners) - 1 if result.feasible_runners else 0
    print(f"{job_name:<15} {status:<12} {best:<25} +{alts} more")

---

## 4️⃣ Visualization

Let's visualize the runner-capability relationships.

In [ ]:
# Optional: Install visualization libraries
try:
    import matplotlib.pyplot as plt
    import networkx as nx
    VIZ_AVAILABLE = True
except ImportError:
    print("⚠️ Install matplotlib and networkx for visualizations:")
    print("   pip install matplotlib networkx")
    VIZ_AVAILABLE = False

In [ ]:
if VIZ_AVAILABLE:
    # Create capability graph
    G = nx.Graph()
    
    # Add runner nodes
    for name, runner in onto.runners.items():
        color = "lightgreen" if runner.online else "lightcoral"
        G.add_node(name, node_type="runner", color=color)
    
    # Add capability nodes and edges
    all_caps = set()
    for runner in onto.runners.values():
        for cap in runner.capabilities:
            all_caps.add(cap.name)
            G.add_node(cap.name, node_type="capability", color="lightblue")
            G.add_edge(runner.name, cap.name)
    
    # Draw
    plt.figure(figsize=(14, 10))
    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)
    
    # Draw runners
    runner_nodes = [n for n, d in G.nodes(data=True) if d.get('node_type') == 'runner']
    runner_colors = [G.nodes[n]['color'] for n in runner_nodes]
    nx.draw_networkx_nodes(G, pos, nodelist=runner_nodes, node_color=runner_colors,
                           node_size=2000, node_shape='s')
    
    # Draw capabilities
    cap_nodes = [n for n, d in G.nodes(data=True) if d.get('node_type') == 'capability']
    nx.draw_networkx_nodes(G, pos, nodelist=cap_nodes, node_color='lightblue',
                           node_size=1000, node_shape='o')
    
    # Draw edges and labels
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    nx.draw_networkx_labels(G, pos, font_size=8)
    
    plt.title("🧠 NSAI Runner-Capability Graph\n(🟩 = online, 🟥 = offline, 🔵 = capability)")
    plt.axis('off')
    plt.tight_layout()
    plt.show()
else:
    print("Skipping visualization (libraries not installed)")

In [ ]:
if VIZ_AVAILABLE:
    # Feasibility heatmap
    import numpy as np
    
    # Jobs to test
    test_jobs = {
        "docker-only": {"tags": ["docker-any"]},
        "docker+gcp": {"tags": ["docker-any", "gcp"]},
        "gpu-job": {"tags": ["docker", "gpu"]},
        "k8s-deploy": {"tags": ["k8s"]},
        "macos-build": {"tags": ["shell", "macos"]},
        "azure-k8s": {"tags": ["kubernetes", "azure"]},
    }
    
    runner_names = list(onto.runners.keys())
    job_names = list(test_jobs.keys())
    
    # Build matrix
    matrix = np.zeros((len(job_names), len(runner_names)))
    for i, (job_name, job_def) in enumerate(test_jobs.items()):
        result = solver.solve(job_def, job_name, include_offline=True)
        for j, runner_name in enumerate(runner_names):
            if runner_name in result.feasible_runners:
                # Score based on ranking
                for rn, score in result.ranked_runners:
                    if rn == runner_name:
                        matrix[i, j] = score
                        break
    
    # Plot heatmap
    fig, ax = plt.subplots(figsize=(12, 6))
    im = ax.imshow(matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)
    
    ax.set_xticks(range(len(runner_names)))
    ax.set_xticklabels(runner_names, rotation=45, ha='right')
    ax.set_yticks(range(len(job_names)))
    ax.set_yticklabels(job_names)
    
    # Add values
    for i in range(len(job_names)):
        for j in range(len(runner_names)):
            val = matrix[i, j]
            color = 'white' if val > 0.5 else 'black'
            text = f"{val:.1f}" if val > 0 else "✗"
            ax.text(j, i, text, ha='center', va='center', color=color, fontsize=10)
    
    plt.colorbar(im, label='Preference Score')
    plt.title('🎯 Job-Runner Feasibility Matrix\n(0 = infeasible, 1 = perfect match)')
    plt.xlabel('Runners')
    plt.ylabel('Jobs')
    plt.tight_layout()
    plt.show()

---

## 5️⃣ Experiments (Paper Material)

Benchmarks and comparisons for the JKU Bachelor thesis.

In [ ]:
import time
import random

def benchmark_solver(num_runners: int, num_jobs: int, num_caps: int = 5) -> dict:
    """Benchmark CSP solver with synthetic data."""
    # Create ontology with random runners
    onto = RunnerOntology()
    all_caps = ["docker", "shell", "linux", "macos", "windows", 
                "gcp", "aws", "azure", "gpu", "arm64", "kubernetes"]
    
    for i in range(num_runners):
        caps = random.sample(all_caps, min(num_caps, len(all_caps)))
        onto.add_runner(f"runner-{i}", capabilities=caps)
    
    # Create solver
    solver = ConstraintSolver(onto, JobRequirementParser())
    
    # Generate random jobs
    jobs = {}
    for i in range(num_jobs):
        num_required = random.randint(1, 3)
        tags = random.sample(all_caps, num_required)
        jobs[f"job-{i}"] = {"tags": tags}
    
    # Benchmark
    start = time.perf_counter()
    results = solver.solve_batch(jobs)
    elapsed = (time.perf_counter() - start) * 1000
    
    # Stats
    feasible = sum(1 for r in results.values() if r.is_feasible)
    
    return {
        "runners": num_runners,
        "jobs": num_jobs,
        "total_ms": elapsed,
        "per_job_ms": elapsed / num_jobs,
        "feasible_rate": feasible / num_jobs
    }

print("🔬 Running benchmarks...\n")
print(f"{'Runners':<10} {'Jobs':<10} {'Total (ms)':<12} {'Per Job (ms)':<15} {'Feasible %'}")
print("-" * 60)

for num_runners in [5, 10, 50, 100]:
    for num_jobs in [10, 100, 1000]:
        result = benchmark_solver(num_runners, num_jobs)
        print(f"{result['runners']:<10} {result['jobs']:<10} "
              f"{result['total_ms']:<12.2f} {result['per_job_ms']:<15.4f} "
              f"{result['feasible_rate']*100:.1f}%")

In [ ]:
# Comparison: With vs Without Symbolic Filtering
print("\n📊 Symbolic Filtering Impact\n")
print("Scenario: 100 runners, job requires [docker, gpu]\n")

# Setup
onto_large = RunnerOntology()
for i in range(95):  # 95 regular runners
    onto_large.add_runner(f"runner-{i}", capabilities=["docker", "linux"])
for i in range(5):   # 5 GPU runners
    onto_large.add_runner(f"gpu-runner-{i}", capabilities=["docker", "linux", "gpu"])

solver_large = ConstraintSolver(onto_large, JobRequirementParser())
result = solver_large.solve({"tags": ["docker"], "image": "nvidia/cuda:11.8"}, "gpu-job")

print(f"Total runners:     {len(onto_large.runners)}")
print(f"Feasible runners:  {len(result.feasible_runners)}")
print(f"Pruned runners:    {len(result.pruned_runners)}")
print(f"\n→ MAB only needs to explore {len(result.feasible_runners)} options instead of {len(onto_large.runners)}!")
print(f"→ Search space reduced by {(1 - len(result.feasible_runners)/len(onto_large.runners))*100:.1f}%")

---

## 6️⃣ Playground

Experiment with your own runners and jobs!

In [ ]:
# 🎮 PLAYGROUND: Modify and experiment!

# Create your own ontology
my_onto = RunnerOntology()

# Add your runners here:
my_onto.add_runner("my-runner-1", capabilities=["docker", "linux"])
my_onto.add_runner("my-runner-2", capabilities=["shell", "macos"])
# Add more...

# Create solver
my_solver = ConstraintSolver(my_onto, JobRequirementParser())

# Test a job
my_job = {
    "tags": ["docker-any"],
    "image": "python:3.11"
}

result = my_solver.solve(my_job, "my-test-job")
print(result.explanation)

In [ ]:
# 🎮 PLAYGROUND: Test with real .gitlab-ci.yml

my_yaml = """
# Paste your .gitlab-ci.yml content here!

test:
  tags:
    - docker-any
  image: python:3.11
  script:
    - echo "Hello"
"""

my_jobs = parser.parse_yaml(my_yaml)
for name, reqs in my_jobs.items():
    print(f"{name}: {reqs.to_dict()}")

---

## 7️⃣ Production Ontology

Load the actual blauweiss production configuration.

In [ ]:
# Load production ontology
prod_onto = create_blauweiss_ontology()

print("🏭 Production Ontology:\n")
for name, runner in prod_onto.runners.items():
    status = "🟢 online" if runner.online else "🔴 offline"
    caps = [c.name for c in runner.capabilities]
    print(f"{name}:")
    print(f"  Status: {status}")
    print(f"  Capabilities: {caps}")
    print(f"  Cost: €{runner.cost_per_minute}/min")
    print()

In [ ]:
# Serialize for inspection
print("📄 Ontology as JSON:\n")
print(prod_onto.to_json(indent=2))

---

## 🎓 Next Steps

1. **Integration with MAB**: Connect to `runner_bandit` service
2. **Real GitLab Webhooks**: Process actual job events
3. **Paper Experiments**: More benchmarks, baselines
4. **OWL Export**: Full semantic web compatibility

---

*NSAI - Neurosymbolic AI Runner Selection*  
*Blauweiss LLC | JKU AI Bachelor 2026*